## Amazon Bedrock Text-to-SQL (Tanner)

### Intro and Goal
This Jupyter Notebook is designed to illustrate a few-shot Text-to-SQL approach on the Northwind database.

The goal is to take a user prompt along with a SQL database schema, supplement the prompt the prompt with relevant samples, and then generate a corresponding SQL query.

### Steps
1. Download SQL schema
2. Download ground truth dataset comprised of questions and SQL queries for a given database (e.g. Northwind)
3. Generate and run SQL queries

In [ ]:
# 1. Create a python environment

# !conda create -y --name bedrock-router-eval python=3.11.8
# !conda init && activate bedrock-router-eval
# !conda install -n bedrock-router-eval ipykernel --update-deps --force-reinstall -y
# !conda install -c conda-forge ipython-sql

## OR
# !python -m venv venv
# !source venv/bin/activate  # On Windows, use `venv\Scripts\activate`

In [ ]:
# 2. Install dependencies

# !pip install -r requirements.txt

### Set Environment Variables

In [ ]:
# 3. Import necessary libraries and load environment variables
import numpy as np
from scipy.spatial.distance import cdist
import json
from dotenv import load_dotenv, find_dotenv
import os
import boto3
import sqlite3
from pandas.io import sql
from botocore.config import Config
import pandas as pd
import io
import json
from io import StringIO
import sqlparse
import sqlite3
import time
import matplotlib.pyplot as plt
import re
import typing as t
from queue import Queue
from threading import Thread
from concurrent.futures import ThreadPoolExecutor, as_completed

# loading environment variables that are stored in local file
local_env_filename = 'dev.env'
load_dotenv(find_dotenv(local_env_filename),override=True)

os.environ['REGION'] = os.getenv('REGION')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
os.environ['SQL_DATABASE'] = os.getenv('SQL_DATABASE') # LOCAL, SQLALCHEMY, REDSHIFT
os.environ['SQL_DIALECT'] = os.getenv('SQL_DIALECT') # SQlite, PostgreSQL

REGION = os.environ['REGION']
HF_TOKEN = os.environ['HF_TOKEN']
SQL_DATABASE = os.environ['SQL_DATABASE']
SQL_DIALECT = os.environ['SQL_DIALECT']